# T9 - Advanced features

This tutorial covers advanced features of Covasim, including custom population options and changing the internal computational methods.

## Defining populations with SynthPops

For complex populations, we suggest using [SynthPops](http://synthpops.org), a Python library designed specifically for this purpose. In contrast the population methods built-in to Covasim, SynthPops uses data to produce synthetic populations that are statistically indistinguishable from real ones. For a relaively complex example of how SynthPops was used to create a complex school network for the Seattle region, see [here](https://github.com/institutefordiseasemodeling/testing-the-waters/blob/main/covasim_schools/school_pop.py).

## Defining layers

TBC

## Defining custom population properties

## Changing Numba options

